In [2]:
import os
import numpy as np
from PIL import Image

In [3]:
def train_validation_test_split(raw_data, annotated_data, train_percent=0.8, validation_percent=0.1, test_percent=0.1):
  assert train_percent + validation_percent + test_percent == 1.0
  raw_files = os.listdir(raw_data)
  annotated_files = os.listdir(annotated_data)
  assert len(raw_files) == len(annotated_files)
  num_files = len(raw_files)

  train_indices = np.random.choice(num_files, size=int(num_files * train_percent), replace=False)
  remaining_indices = np.setdiff1d(np.arange(num_files), train_indices)
  validation_indices = np.random.choice(remaining_indices, size=int(num_files * validation_percent), replace=False)
  test_indices = np.setdiff1d(remaining_indices, validation_indices)
  
  for raw, ann in zip(sorted(raw_files), sorted(annotated_files)):
    raw_index = int(raw.split('.')[0].split('_')[1])
    ann_index = int(ann.split('.')[0].split('_')[1])
    assert raw_index == ann_index
    if (raw_index not in train_indices) and (raw_index not in validation_indices) and (raw_index not in test_indices):
      raise ValueError('Index not found in any set')
    
  return train_indices, validation_indices, test_indices

train, validate, test = train_validation_test_split('_outRaw', '_outSeg')

In [4]:
def make_odgt(raw_data, annotated_data, train_idx, validate_idx, test_idx, out_dir):
  raw_files = os.listdir(raw_data)
  annotated_files = os.listdir(annotated_data)
  assert len(raw_files) == len(annotated_files)
  for raw, ann in zip(sorted(raw_files), sorted(annotated_files)):
    raw_index = int(raw.split('.')[0].split('_')[1])
    ann_index = int(ann.split('.')[0].split('_')[1])
    assert raw_index == ann_index

    raw = Image.open(raw)
    ann = Image.open(ann)
    width, height = raw.size
    ann_width, ann_height = ann.size
    assert width == ann_width
    assert height == ann_height
    
    raw_path = os.path.join(raw_data, raw)
    ann_path = os.path.join(annotated_data, ann)
    odgt_line = {"path_img": raw_path, "fpath_segm": ann_path, "width": {width}, "height": {height}}
    if raw_index in train_idx:
      with open(os.path.join(out_dir, 'train.odgt'), 'a') as f:
        f.write(str(odgt_line) + '\n')
    elif raw_index in validate_idx:
      with open(os.path.join(out_dir, 'validate.odgt'), 'a') as f:
        f.write(str(odgt_line) + '\n')
    elif raw_index in test_idx:
      with open(os.path.join(out_dir, 'test.odgt'), 'a') as f:
        f.write(str(odgt_line) + '\n')
    
make_odgt('_outRaw', '_outSeg', train, validate, test, 'odgt')

FileNotFoundError: [Errno 2] No such file or directory: '/Data/toledod/collectedData/trafficPath/raw_0.png'